In [4]:
import pandas as pd
from pathlib import Path

dataset_path = Path("/mnt/i/bio/mave_curation/Kotler_TP53_RelativeFitnessHCT116/")

In [5]:
data = pd.read_excel(dataset_path / "raw" / "mmc5.xlsx")

In [6]:
data[data.AA_change.isna()]

,Number,Var_seq,Identifier,SubLib,Backbone,Mut_type,Position,Seq_change,Codon_num,AA_change,Sec_mut_position,Sec_seq_change,Sec_codon_num,Sec_AA_change,MutID,IARC_Desc,Silent,RFS_HCT116
41,6460,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400078,DBDD,wt,wtp53_DBDD_seq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,-1.146098
95,6514,ATGATTCCCAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400132,DBDD,wt,IARC.InDel.GeneVars,Unspecified,NaN,247.0,NaN,NaN,NaN,NaN,NaN,MutID6136,del5insTTCCC,False,0.059228
96,6515,ATGCCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCAGT...,400133,DBDD,wt,IARC.InDel.GeneVars,Unspecified,NaN,247.0,NaN,NaN,NaN,NaN,NaN,MutID3272,del2,False,0.119382
97,6516,ATGGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCAGTGG...,400134,DBDD,wt,IARC.InDel.GeneVars,Unspecified,NaN,247.0,NaN,NaN,NaN,NaN,NaN,MutID3273,del4,False,0.543468
455,6916,ATGAACCGGGCCCATCCTCACCATCATCACACTGGAAGACTCCAGT...,400534,DBDD,wt,IARC.InDel.GeneVars,Unspecified,NaN,248.0,NaN,NaN,NaN,NaN,NaN,MutID3302,del2,False,0.029183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109,9656,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403274,DBDD,wt,IARC.InDel.GeneVars,Unspecified,NaN,287.0,NaN,NaN,NaN,NaN,NaN,MutID3962,del5,False,-0.335658
3143,9691,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403309,DBDD,wt,IARC.InDel.GeneVars,Unspecified,NaN,288.0,NaN,NaN,NaN,NaN,NaN,MutID5782,del2,False,-0.349134
3144,9693,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403311,DBDD,wt,IARC.InDel.GeneVars,Unspecified,NaN,288.0,NaN,NaN,NaN,NaN,NaN,MutID3973,del2,False,-0.639609
3189,9799,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403417,DBDD,wt,IARC.InDel.GeneVars,Unspecified,NaN,291.0,NaN,NaN,NaN,NaN,NaN,MutID4025,del2,False,0.184177


In [22]:
subset = data[(~data.AA_change.isna()) & (data.AA_change.str.len() == 3) & (data.Sec_AA_change.isna()) & (data.Mut_type != "DelAA")]

In [23]:
subset

,Number,Var_seq,Identifier,SubLib,Backbone,Mut_type,Position,Seq_change,Codon_num,AA_change,Sec_mut_position,Sec_seq_change,Sec_codon_num,Sec_AA_change,MutID,IARC_Desc,Silent,RFS_HCT116
26,6445,AATAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400063,DBDD,wt,AASub,736,ATG>AAT,246.0,M>N,NaN,NaN,NaN,NaN,NaN,NaN,False,0.157110
27,6446,AGTAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400064,DBDD,wt,AASub,736,ATG>AGT,246.0,M>S,NaN,NaN,NaN,NaN,NaN,NaN,False,0.422444
28,6447,CAGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400065,DBDD,wt,AASub,736,ATG>CAG,246.0,M>Q,NaN,NaN,NaN,NaN,NaN,NaN,False,0.439412
29,6448,CCGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400066,DBDD,wt,AASub,736,ATG>CCG,246.0,M>P,NaN,NaN,NaN,NaN,NaN,NaN,False,-0.247905
30,6449,GAGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400067,DBDD,wt,AASub,736,ATG>GAG,246.0,M>E,NaN,NaN,NaN,NaN,NaN,NaN,False,0.314707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3208,9821,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403439,DBDD,wt,Sub,875,A>C,292.0,K>T,NaN,NaN,NaN,NaN,NaN,NaN,False,-0.919863
3209,9822,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403440,DBDD,wt,Sub,875,A>G,292.0,K>R,NaN,NaN,NaN,NaN,NaN,NaN,False,-1.260542
3210,9823,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403441,DBDD,wt,Sub,875,A>T,292.0,K>I,NaN,NaN,NaN,NaN,NaN,NaN,False,-0.790598
3214,9827,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403445,DBDD,wt,Sub,876,A>C,292.0,K>N,NaN,NaN,NaN,NaN,NaN,NaN,False,-0.985578


In [24]:
from Bio.PDB.Polypeptide import protein_letters_3to1
protein_letters_1to3 = {v: k.title() for k, v in protein_letters_3to1.items()}
protein_letters_1to3["*"] = "Ter"
def make_hgvs_pro(r):
    return f"{protein_letters_1to3[r.AA_change[0]]}{r.Position}{protein_letters_1to3[r.AA_change[-1]]}"

In [25]:
subset = subset.assign(hgvs_pro=subset.apply(make_hgvs_pro, axis=1))

In [27]:
subset = subset.rename(columns={"RFS_HCT116":"score"})

In [28]:
subset

,Number,Var_seq,Identifier,SubLib,Backbone,Mut_type,Position,Seq_change,Codon_num,AA_change,Sec_mut_position,Sec_seq_change,Sec_codon_num,Sec_AA_change,MutID,IARC_Desc,Silent,score,hgvs_pro
26,6445,AATAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400063,DBDD,wt,AASub,736,ATG>AAT,246.0,M>N,NaN,NaN,NaN,NaN,NaN,NaN,False,0.157110,Met736Asn
27,6446,AGTAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400064,DBDD,wt,AASub,736,ATG>AGT,246.0,M>S,NaN,NaN,NaN,NaN,NaN,NaN,False,0.422444,Met736Ser
28,6447,CAGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400065,DBDD,wt,AASub,736,ATG>CAG,246.0,M>Q,NaN,NaN,NaN,NaN,NaN,NaN,False,0.439412,Met736Gln
29,6448,CCGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400066,DBDD,wt,AASub,736,ATG>CCG,246.0,M>P,NaN,NaN,NaN,NaN,NaN,NaN,False,-0.247905,Met736Pro
30,6449,GAGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,400067,DBDD,wt,AASub,736,ATG>GAG,246.0,M>E,NaN,NaN,NaN,NaN,NaN,NaN,False,0.314707,Met736Glu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3208,9821,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403439,DBDD,wt,Sub,875,A>C,292.0,K>T,NaN,NaN,NaN,NaN,NaN,NaN,False,-0.919863,Lys875Thr
3209,9822,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403440,DBDD,wt,Sub,875,A>G,292.0,K>R,NaN,NaN,NaN,NaN,NaN,NaN,False,-1.260542,Lys875Arg
3210,9823,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403441,DBDD,wt,Sub,875,A>T,292.0,K>I,NaN,NaN,NaN,NaN,NaN,NaN,False,-0.790598,Lys875Ile
3214,9827,ATGAACCGGAGGCCCATCCTCACCATCATCACACTGGAAGACTCCA...,403445,DBDD,wt,Sub,876,A>C,292.0,K>N,NaN,NaN,NaN,NaN,NaN,NaN,False,-0.985578,Lys876Asn


In [29]:
subset.to_csv(dataset_path / "scoreset.csv", index=False)